## Newsletter Email agents with Handoffs
##### Part 1 (Tools)
- Create 3 agents which will write the emails.
- Convert them into tools also create another tools which will send the push notification


##### Part 2 (Handoff)
- Create 2 agents
    1. Generate a catchy subject line
    2. Convert the email into an HTML.
- Create a Handoff agent  
    This agent will be the email formatter and sender agent.  
    This agent will use the subject_tool, html_tool and send_push_notification as tools
    **NOTE: This agent will have an handoff instruction**
        This instrution will specify what the agent is going to do

##### Part 3 (Sales Manager)
- Create a new agent which will be the Sales Manager
    - Responsible to use all the tools and Handoffs
    - tools -> agent_tools from part 1
    - handoffs -> Part2 handoff agent (will use the subject_tool and html_tool internally)

All this will work cohisively to create a well formatted email and send it to the customer.

In [14]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
import os
import requests

In [15]:
load_dotenv(override=True)

True

### Part 1 (Tools)

In [16]:
# PART 1 tools
intruction_1 = """You are a sales agent working for FinAI, a company that provides AI solutions for financial institutions.
You write professional, serious cold emails.
"""
intruction_2 = """You are a humorous, engaging sales agent working for FinAI, a company that provides AI solutions for financial institutions.
You write witty, engaging cols emails that are likely to get a response
"""
intruction_3 = """You are a busy sales agent working for FinAI, a company that provides AI solutions for financial institutions.
You write professional, serious cold emails.
"""
sales_agent_1 = Agent(instructions=intruction_1, name="Professional Sales Agent", model="gpt-4o-mini")
sales_agent_2 = Agent(instructions=intruction_2, name="Humorous Sales Agent", model="gpt-4o-mini")
sales_agent_3 = Agent(instructions=intruction_3, name="Busy Sales Agent", model="gpt-4o-mini")
description = "Write a cold email addressed to 'Dear Bank Manager'"
tool1 = sales_agent_1.as_tool(tool_name="sales_agent_1", tool_description=description)
tool2 = sales_agent_2.as_tool(tool_name="sales_agent_2", tool_description=description)
tool3 = sales_agent_3.as_tool(tool_name="sales_agent_3", tool_description=description)

In [17]:
# Push notification helper function
def push(text):
    requests.post(
        "https://api.pushover.net/1/messages.json",
        data={
            "token": os.getenv("PUSHOVER_TOKEN"),
            "user": os.getenv("PUSHOVER_USER"),
            "message": text,
        }
    )

In [18]:
# Tool which send the push notification
@function_tool
def send_push_notification(message: str) -> None:
    """Send a push notification with the given message."""
    push(message)
    return {"status": "success"}

### Part 2 (Handoffs)

In [19]:
# Part 2 (Handoff)

subject_instruction = "Generate a concise, engaging email subject line for a cold sales email. \
    You are given a message and you are needed to generate a subject line for it."
    
html_instruction = "Generate a professional HTML email body for a cold sales email. \
    You are given a text email body which might have some markdown \
    and you need to convert it to an HTML email body with simple, clear, compelling layout and design."
    
subject_write = Agent(
    instructions=subject_instruction, name="Subject Writer", model="gpt-4o-mini"
)

html_converter = Agent(
    instructions=html_instruction, name="HTML Converter", model="gpt-4o-mini"
)

subject_writer_tool = subject_write.as_tool(tool_name="subject_writer", tool_description=subject_instruction)
html_converter_tool = html_converter.as_tool(tool_name="html_converter", tool_description=html_instruction)



In [20]:
# Handoff push notification agent
push_notification_instruction = """You are an email formatter and sender. You receive the bosy of an email to be sent.
You first user the subject_writer_tool to write a subject for the email, then use the html_converter_tool to convert the body to HTML.
Finally, you use the send_push_notification tool to send a push notification with the subject and HTML body as text.
"""

handoff_push_agent = Agent(
    name="Handoff Push Notification Agent",
    instructions=push_notification_instruction,
    tools=[subject_writer_tool, html_converter_tool, send_push_notification],
    model="gpt-4o-mini",
    handoff_description="Conver the email to HTML and send it as a push notification"
)


### Part 3 (Sales Manager)

In [22]:
# PART 3

sales_manager_instructions = """
You are a sales manager at FinAI, a company that provides AI solutions for financial institutions.
Your goal is to find the single best cols smail using sales_sgent tools.

Follw this steps carefully:
1. Generate Drafts: Use all three sales_aegnt tools to generate three different cold email drafts. Do no proceed until you have three drafts.

2. Evaluate and Select: Review the drafst and choose the single best email using your judgement of which on ei the most professional and likely to get a response.
You can use the tools multiple times if you're not satisfied witht he results from the first try.

3. Handoff for Sending: Pass Only the winning email draft to the handoff_push_agent(Handoff Push Notification Agent) for formatting and sending.
The handoff_push_agent will take care of the formatting and sending.

Crucial Rules:
- You must use the sales agent tools to generate the drafts - do not write them yourself.
- You must hand off exactly ONE email ot the handoff_push_agent(Handoff Push Notification Agent) - never more than one.
"""

sales_manager_agent = Agent(
    name = "Sales Manager Agent",
    instructions = sales_manager_instructions,
    tools=[tool1, tool2, tool3],
    model="gpt-4o-mini",
    handoffs=[handoff_push_agent]
)

message = "Send a cold sales email address to Dead Bank Manager from Ashutosh"

with trace("Sales Manager"):
    result = await Runner.run(sales_manager_agent, message)
